In [4]:
# PyTorch @Sourcence 15-11-2018
#
# libreria di ML/DL seconda a tensorflow
# nella 2.0 hanno aggiunto l'ottimizzazione nel deploy
# tensorflow è sterminato, pytorch è più semplice da imparare e ti orienta meglio il begineer
# tensorflow carica i dati in molto più potente e flessibile
# sviluppata da Facebook, che la utilizzava insieme a Caffe2, e poi sono state unite
# con pip porto meglio il progetto
# con anaconda gestisco meglio le dipendenze
# operazioni sui tensori
# il cuore sono matrici di numeri
# come numpy

# torchvision è una libreria che aiuta per image classification

In [7]:
# numpy
import numpy as np
x = np.zeros((3,2))
print(x)
# di default float64

[[0. 0.]
 [0. 0.]
 [0. 0.]]


In [8]:
# tensore in pytorch: come numpy
y = np.ones((3,2))
print(y)
# a differenze di tensorflow, in cui l'elaborazione è separata e non dà subito il risultato
# pytorch dà subito il risultato perchè è più orientato a sperimentare

[[1. 1.]
 [1. 1.]
 [1. 1.]]


In [15]:
print(y.shape)
print(y.size)

(3, 2)
6


In [17]:
# indicizzazione di tensori
print(y[1,1])
print(y[2])

1.0
[1. 1.]


In [18]:
# attenzione alle funzioni di trasformazione da array di numpy a array di pytorch e viceversa
xx = torch.from_numpy(x)
# attenzione: se voglio sommare 1 a tutti gli elementi della matrice
xx += 1
x
# se trasformo la partenza (x), sto trasformando anche l'arrivo (xx) e viceversa
# questo perchè in realtà l'oggetto (array) di pytorch è una sorta di wrapper dell'oggetto (array) di numpy: 
# alla fine è un solo oggetto

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [22]:
y[0,0].item()

1.0

In [24]:
print(x.mean())
print(x.mean(axis=0))

1.0
[1. 1.]


In [25]:
# fin qui abbiamo capito che la parte base di pytorch è esattamente numpy
# pytorch inizia a differenziarsi per come calcola derivate e gradient

In [26]:
v = torch.rand(4, requires_grad=True)
# ho definito un tensore
# quando faccio back propagation di NN ho bisogno di ottimizzare alcuni paramentrio: con requires_grad dico
# a pytorch che questo sarà uno di quei paramentri di cui a un certo punto avrò bisogno di chiederne
# il gradiente...pytorch lo traccerà

# voglio i numeri del tensore
print(v.data)

tensor([0.6215, 0.3962, 0.6425, 0.4391])


In [27]:
# pytorch si può ricostruire, e mostrare, tutte le operazioni fatte nell'ordine in cui sono state fatte (tape)
z = torch.sqrt(v + 3)
z.grad_fn

In [28]:
z = torch.sum(torch.sqrt(v + 3))
# backward srotola il nastro, ripercorre le operazioni, e salva dentro la proprietà grad questo modello
# così posso richiederlo ogni volta che avrò bisogno
z.backward()
# chiedo il gradiente di z su v, mi dà il tensore:
v.grad
# non posso fare il backwad due volte

tensor([0.2627, 0.2713, 0.2620, 0.2696])

In [29]:
# parametri di cui tipicamente non ho bisogno che siano tracciati, ad es. l'accuratezza del modello, 
# posso dire a pytorch di non tracciarli

In [32]:
# Dataset Iris
# lo carichiamo
# datasets è molto nice per i primi esperimenti
from sklearn import datasets
data = datasets.load_iris()

In [33]:
# data è un dizionario
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [38]:
# input features X e target y
# X è un array numpy di 150 righe/misurazioni e 4 colonne/grandezze misurate
X = data['data']
y = data['target']
# y è un array di 150 elementi che attengono a tre classi, alle tre classi a cui appartengono gli input/fiori
# il problema di classificazione: dato un fiore, predire a quale classe appartiene
print(X.shape)
print(y)

(150, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [39]:
# orea divdiamo il dataset i parte di training (75%) e parte di test (25%), per evitare overfitting
# in sklearn c'è un modulo, model selection, e dentro c'è la funzione comoda
from sklearn import model_selection
X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y, stratify=y)
# con questo dataset così semplice è facile raggiungere il 100% di accuratezza
# stratify dice di stratificare i subset rispetto alle tre classi target, ossia di cerarmi 30% di casi di classe 1
# 30% di casi di classe 2 e 30% di classe 3

In [41]:
# regressione lineare: modello più semplice per descrivere y a partire da X
# logistic 
w = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(3, requires_grad=True)
def linear(x):
    return x.mm(w) + b
# (*)
# mm matrix multiplication

In [42]:
# pytorch però consente di lavorare a livello più alto, evitando i precedenti dettagli, e utilizzando 
# direttamente i modelli
# torch.nn è il modulo dove trovare tutte le classi
# https://pytorch.org/docs/stable/nn.html
# torch.nn.Linear è priprio il modello che fa tutto quello che stavo facendo poco fa
# torch.nn.Module:
# molto comodo avere quelle due funzioni principali: nella prima (inizializzazione) definisco tutto quello 
# che  mi serve, la seconda (forward) prende l'inizializzazione e l'input (x) e fornisce l'output
# scrivo classi custom che estende la classe torch.nn.Module

In [54]:
# il module incapsula tutta la logica per allenarlo
class ourModel(torch.nn.Module):
# in python l'inizializzazione si fa così e, a differenza di java, va passato esplic self, ossia l'oggetto
# init è il costruttore di un modulo che incapsula un altro modulo
    def __init__(self):
        super(ourModel, self).__init__()
        # qui inizializzo i tensori ossia l'equivalente di (*)
        self.lin = torch.nn.Linear(4, 3)
        
    def forward(self, x):
        # la seguente è come 
        return self.lin(x)
# quindi abbiamo scritto l'analogo di quanto visto in 39 e 41 in forma più chiara e estensibile

In [55]:
# ora possiamo istanziare la classe sul ns caso
nn = ourModel()
nn(torch.from_numpy(X_train[0:3]).float())

tensor([[-0.1010,  0.3429, -0.3320],
        [-0.9605,  1.0214,  0.7831],
        [-0.3549,  1.0196,  0.3372]], grad_fn=<ThAddmmBackward>)

In [58]:
# generatori python
[p for p in nn.parameters()]
# tiro fuoi tuti i parametri che voglio ottimizzare
# un parameter è un tensore con un requires, dentro un modulo

[Parameter containing:
 tensor([[ 0.1185, -0.1747, -0.0989, -0.2680],
         [ 0.0706,  0.0055,  0.2898, -0.3200],
         [-0.1229,  0.0927,  0.3282, -0.0454]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0837, -0.2971, -0.4868], requires_grad=True)]

In [63]:
# qui il modello ha sparato a caso, vedremo come affinare il modello
# definiamo la fujzione costo
# esistono tanti modi di definire l'errore per le diverse cose che si vogliono fare
# in questo tipo di problemi di classificazione uso cross entropia
# loss è il costo
def loss(X, y):
    ypred = nn(x)
    l = torch.nn.CrossEntropyLoss()
    return l(ypred,y)
# ottimizzando loss migliorerà il modello predittivo
# dentro nn troverò loss per varie casistiche (predire valori continui, ecc..)

In [64]:
# come ottimizzo il modello?
# scelgo il numero di iterazioni che voglio fare
#  a ogni iterazione calcolo la loss, calolo il gradiente e iterativamente miglioro i pesi
# e quindi la loss
# scelgo, tra i tanti, di usare l'algoritmo Adam
opt = torch.optim.Adam(params= nn.parameters())
# il parametro "params= nn.parameters()" dichiara quello che voglio ottimizzare

In [65]:
# definisco il numero di iterazioni
iters = 100
# tendenzialmente, se ottimizzo il gradiente sto diminuendo il costo e quindi migliorando 
# l'affidabilità predittiva del modello
X_train_t = torch.from_numpy(X_train).float()
y_train_t = torch.from_numpy(y_train)

# nel vettore seguente ci salviamo i vari valori di loss durante l'ottimizzazzione
all_losses = np.zeroes(iters)

In [62]:
for i in range(iters):
    l = loss(X_train_t, y_train_t)
    allo_losses[i] = l.detach().item()
    # salvo i gradienti
    l.backward()
    # a differenza di tutte le altre librerie, pytorch separa la... da...
    # loop su tutti i parametri e ognuno lo ottimizza
    opt.step()
    # azzera i gradienti di questo ciclo, tanto li ricaloca al successivo
    opt.zero_grad()

AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [ ]:
# questo era il ciclo classico dell'ottimizzazione
# l'obiettivo è vedere se la loss decresce
# ci salviamo i valori di l e li grafichiamo
import matplotlib.pyplot as plt
plt.plot(all_losses)
plt.show()